In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 1. Registro de fornecedores de ingredientes

fornecedores = {}

def registrar_fornecedor(nome, cnpj, produto):
    fornecedores[nome] = {'CNPJ': cnpj, 'Produto': produto}

# Exemplo de registro de fornecedores
registrar_fornecedor('Fornecedor A', '12.345.678/0001-99', 'Queijo')
registrar_fornecedor('Fornecedor B', '98.765.432/0001-00', 'Tomate')

# Visualizando fornecedores registrados
fornecedores

In [ ]:
# 2. Controle de estoque

estoque = {}

def adicionar_estoque(ingrediente, quantidade):
    if ingrediente in estoque:
        estoque[ingrediente] += quantidade
    else:
        estoque[ingrediente] = quantidade

def remover_estoque(ingrediente, quantidade):
    if ingrediente in estoque and estoque[ingrediente] >= quantidade:
        estoque[ingrediente] -= quantidade
    else:
        print(f"Estoque insuficiente de {ingrediente}")

# Exemplo de adição de estoque
adicionar_estoque('Queijo', 10)
adicionar_estoque('Tomate', 20)
adicionar_estoque('Calabresa', 15)
adicionar_estoque('Pepperoni', 10)

# Visualizando estoque atual
estoque

In [ ]:
# 3. Sistema de pedidos online

menu = {
    'Margarita': {'ingredientes': ['Queijo', 'Tomate'], 'preco': 30.0},
    'Pepperoni': {'ingredientes': ['Queijo', 'Tomate', 'Pepperoni'], 'preco': 35.0},
    'Calabresa': {'ingredientes': ['Queijo', 'Tomate', 'Calabresa'], 'preco': 33.0}
}

# Lista para armazenar pedidos
pedidos = []

def mostrar_menu():
    print("MENU DE PIZZAS:")
    for pizza, detalhes in menu.items():
        print(f"{pizza}: R${detalhes['preco']}")

def verificar_estoque(pizza):
    ingredientes_necessarios = menu[pizza]['ingredientes']
    for ingrediente in ingredientes_necessarios:
        if estoque.get(ingrediente, 0) == 0:
            return False
    return True

def fazer_pedido():
    nome = input("Nome: ").upper()
    genero = input("Gênero: ").upper()
    estado_civil = input("Estado Civil: ").upper()
    faixa_etaria = input("Faixa Etária: ").upper()
    endereco = input("Endereço: ").upper()
    telefone = input("Telefone: ").upper()
    quantidade = int(input("Quantas pizzas: "))
    mostrar_menu()
    sabores = []
    for _ in range(quantidade):
        sabor = input("Sabor: ").capitalize()
        if verificar_estoque(sabor):
            sabores.append(sabor)
            for ingrediente in menu[sabor]['ingredientes']:
                remover_estoque(ingrediente, 1)
        else:
            print(f"{sabor} não está disponível no momento.")
    
    tamanho = input("Tamanho: ").upper()
    
    # Emitindo o comprovante do pedido
    comprovante = f"""
    COMPROVANTE DO PEDIDO
    Nome: {nome}
    Gênero: {genero}
    Estado Civil: {estado_civil}
    Faixa Etária: {faixa_etaria}
    Endereço: {endereco}
    Telefone: {telefone}
    Quantidade de Pizzas: {quantidade}
    Sabores: {', '.join(sabores)}
    Tamanho: {tamanho}
    """
    print(comprovante)
    
    # Registrando o pedido
    pedidos.append({
        'nome': nome,
        'genero': genero,
        'estado_civil': estado_civil,
        'faixa_etaria': faixa_etaria,
        'endereco': endereco,
        'telefone': telefone,
        'quantidade': quantidade,
        'sabores': sabores,
        'tamanho': tamanho
    })

# Funções de CSV
def salvar_pedidos_em_csv(nome_arquivo='pedidos.csv'):
    with open(nome_arquivo, 'w', newline='') as csvfile:
        fieldnames = ['nome', 'genero', 'estado_civil', 'faixa_etaria', 'endereco', 'telefone', 'quantidade', 'sabores', 'tamanho']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for pedido in pedidos:
            writer.writerow({
                'nome': pedido['nome'],
                'genero': pedido['genero'],
                'estado_civil': pedido['estado_civil'],
                'faixa_etaria': pedido['faixa_etaria'],
                'endereco': pedido['endereco'],
                'telefone': pedido['telefone'],
                'quantidade': pedido['quantidade'],
                'sabores': ';'.join(pedido['sabores']),  # Convertendo a lista de sabores em uma string separada por ponto e vírgula
                'tamanho': pedido['tamanho']
            })

def restaurar_pedidos_de_csv(nome_arquivo='pedidos.csv'):
    global pedidos
    try:
        pedidos_df = pd.read_csv(nome_arquivo)
        pedidos_df['sabores'] = pedidos_df['sabores'].apply(lambda x: x.split(';'))  # Convertendo de volta para lista
        pedidos = pedidos_df.to_dict(orient='records')
    except FileNotFoundError:
        pedidos = []  # Caso o arquivo não exista, inicializa uma lista vazia

# Inicializa a lista de pedidos a partir do arquivo CSV
restaurar_pedidos_de_csv()

In [ ]:
# 4. Relatório de vendas diário

# Função para gerar gráfico e salvar como PDF
def gerar_grafico(pizzas_por_tipo):
    tipos_pizza = list(pizzas_por_tipo.keys())
    unidades_vendidas = list(pizzas_por_tipo.values())

    plt.figure(figsize=(10, 6))
    plt.bar(tipos_pizza, unidades_vendidas, color='skyblue')
    plt.xlabel('Tipo de Pizza')
    plt.ylabel('Total de Unidades Vendidas')
    plt.title('Total de Unidades Vendidas por Tipo de Pizza')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Salvando o gráfico em PDF
    plt.savefig('relatorio_pizzas_vendidas.pdf')
    plt.close()

# Função para gerar relatório
def gerar_relatorio():
    total_vendas = 0
    total_receita = 0
    pizzas_por_tipo = {}
    clientes_por_genero = {'M': 0, 'F': 0}
    clientes_por_estado_civil = {}
    clientes_por_faixa_etaria = {}

    for pedido in pedidos:
        total_vendas += pedido['quantidade']
        
        try:
            total_receita += sum([menu[sabor]['preco'] for sabor in pedido['sabores']])
        except KeyError:
            # Se ocorrer um KeyError, o sabor pode não estar no menu, então o erro é ignorado.
            continue
        
        for sabor in pedido['sabores']:
            if sabor in pizzas_por_tipo:
                pizzas_por_tipo[sabor] += 1
            else:
                pizzas_por_tipo[sabor] = 1
        
        clientes_por_genero[pedido['genero']] += 1
        
        estado_civil = pedido['estado_civil']
        if estado_civil in clientes_por_estado_civil:
            clientes_por_estado_civil[estado_civil] += 1
        else:
            clientes_por_estado_civil[estado_civil] = 1
        
        faixa_etaria = pedido['faixa_etaria']
        if faixa_etaria in clientes_por_faixa_etaria:
            clientes_por_faixa_etaria[faixa_etaria] += 1
        else:
            clientes_por_faixa_etaria[faixa_etaria] = 1

    ticket_medio = total_receita / len(pedidos) if len(pedidos) > 0 else 0
    
    print("RELATÓRIO DE VENDAS DIÁRIO")
    print(f"Total de unidades vendidas: {total_vendas}")
    print(f"Total de receita: R${total_receita:.2f}")
    print(f"Ticket médio: R${ticket_medio:.2f}")
    print(f"Pizzas vendidas por tipo: {pizzas_por_tipo}")
    print(f"Clientes por gênero: {clientes_por_genero}")
    print(f"Clientes por estado civil: {clientes_por_estado_civil}")
    print(f"Clientes por faixa etária: {clientes_por_faixa_etaria}")

    # Gerar e salvar gráfico
    gerar_grafico(pizzas_por_tipo)

In [ ]:
# Repita esses 3 comandos para fazer pedido, gerar relatório com gráfico e salvar os pedidos

fazer_pedido()

gerar_relatorio()

salvar_pedidos_em_csv()